<a href="https://colab.research.google.com/github/ChitranshS/HiringChallenge-1/blob/main/Hiring_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Check GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b47b48ef-e201-9a56-1ef3-cc0e53420212)


In [ ]:
#@title Setup
import os, subprocess

def setup():
    install_cmds = [
        ['pip', 'install', 'gradio'],
        ['pip', 'install', 'open_clip_torch'],
        ['pip', 'install', 'clip-interrogator'],
        ['pip', 'install', 'together'],
        ['pip','install','gradio_client']
    ]
    for cmd in install_cmds:
        print(subprocess.run(cmd, stdout=subprocess.PIPE).stdout.decode('utf-8'))

setup()

import gradio as gr
from clip_interrogator import Config, Interrogator

config = Config()
config.clip_model_name = 'ViT-H-14/laion2b_s32b_b79k'
config.caption_model_name = 'blip-large'
ci = Interrogator(config)


def image_to_prompt(image):
    ci.config.chunk_size =1024
    ci.config.flavor_intermediate_count =1024
    image = image.convert('RGB')
    mode = 'fast'
    generated_prompt = ci.interrogate_fast(image)
    return generated_prompt




In [ ]:
import gradio as gr

def prompt_tab():
    global stored_prompt  # Declare the stored_prompt variable as global

    with gr.Column():
        with gr.Row():
            print(1)
            image = gr.Image(type='pil', label="Image")
            print(2)
        prompt = gr.Textbox(label="Prompt")
        print(3)
        button = gr.Button("Generate prompt")
        print(4)
        button.click(image_to_prompt, inputs=[image], outputs=[prompt])
        print(5)

            # Store the prompt value after rendering
        prompt.change(update_stored_prompt, inputs=[prompt], outputs=[])


def update_stored_prompt(prompt):
    global stored_prompt
    stored_prompt = prompt  # Assign the prompt value to the stored_prompt variable

with gr.Blocks() as ui:
    with gr.Tab("Prompt"):
        stored_prompt = ""  # Initialize the stored_prompt variable
        prompt_tab()
    ui.launch()


In [32]:
stored_prompt = "a pair of headphones on a black background, wearing modern headphone, mars black, sennheiser, giant headphones rocking out, concept art-h 640, official product image, headset, headphones, headphones dj rave, - h 6 4 0, -h 640, wearing headset, bronze headset, product image, highly detailed-h 704"

In [10]:
# !pip install together

In [33]:
import os
import together

together.api_key = "492d06035d1fb29fc36b26ba45d4746734b41f077027213f2b73bf7bee5a64d3"
response = together.Complete.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    prompt=f"""Generate a prompt in paragraphs for creating a background environment without the use of \n or markdown.The product is described as follows:{stored_prompt}. The environment should complement the features and use case of the product, enhancing its appeal in a visually compelling manner. Please include specifics like the
setting, mood, key elements, and any thematic details that align with the product's branding and target audience.You have to send it so that it only has text and return to an api call so don't include \n etc""",
    temperature = 0.7,
    top_k = 50,
    top_p = 0.7,
    repetition_penalty = 1
)
print(response)


<ipython-input-33-20fdcd4d4cfb>:5: DeprecationWarning: Call to deprecated function create.
  response = together.Complete.create(
/usr/local/lib/python3.10/dist-packages/together/legacy/complete.py:23: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment variable instead.
  warnings.warn(API_KEY_WARNING)


{'id': '879fbd715f1b7ba8-ATL', 'object': <ObjectType.Completion: 'text.completion'>, 'created': 1714062874, 'model': 'meta-llama/Llama-3-70b-chat-hf', 'choices': [{'index': 0, 'logprobs': None, 'finish_reason': <FinishReason.Length: 'length'>, 'text': ".\n\nHere is a prompt:\n\nCreate a background environment that complements the features and use case of a pair of modern headphones, specifically the Sennheiser brand, without using markdown or HTML. The environment should enhance the product's appeal in a visually compelling manner. \n\nThe setting is a dimly lit, intimate music studio with a dark, muted color palette. The mood is energetic and creative, evoking a sense of artistic expression. Key elements include a sleek, modern desk with a few scattered music sheets, a microphone, and a laptop. The walls are adorned with inspirational quotes and music-inspired artwork. \n\nThematic details that align with the product's branding and target audience include a subtle, neon-lit cityscape 

In [34]:
generation_prompt = response['choices'][0]


In [13]:
# !pip install gradio_client

In [35]:
from gradio_client import Client

client = Client("https://hysts-controlnet-v1-1.hf.space/")
result = client.predict(
		"./123123.jpg",	# str (filepath on your computer (or URL) of image) in 'parameter_8' Image component
		generation_prompt,	# str  in 'Prompt' Textbox component
		"best quality, extremely detailed",	# str  in 'Additional prompt' Textbox component
		"longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality",	# str  in 'Negative prompt' Textbox component
		1,	# int | float (numeric value between 1 and 1) in 'Number of images' Slider component
		768,	# int | float (numeric value between 256 and 768) in 'Image resolution' Slider component
		20,	# int | float (numeric value between 1 and 100) in 'Number of steps' Slider component
		12,	# int | float (numeric value between 0.1 and 30.0) in 'Guidance scale' Slider component
		0,	# int | float (numeric value between 0 and 2147483647) in 'Seed' Slider component
		100,	# int | float (numeric value between 1 and 255) in 'Canny low threshold' Slider component
		200,	# int | float (numeric value between 1 and 255) in 'Canny high threshold' Slider component
		api_name="/canny"
)
print(result)

Loaded as API: https://hysts-controlnet-v1-1.hf.space/ ✔
/tmp/gradio/213e9684-a9d3-4ab8-92c9-a94409d767b2


In [36]:
from PIL import Image

# Load the image
image = Image.open('../content/input.png')

# Specify the new dimensions
new_width = 512
new_height = 512

# Resize the image
resized_image = image.resize((new_width, new_height), Image.LANCZOS)

# Save the resized image
resized_image.save('resized_image1.png')

# Display the resized image (optional)
resized_image.show()


In [ ]:
import os
for dirpath, dirnames, filenames in os.walk(result):
  if dirnames:
   directories = dirnames
   break
result

In [55]:
from PIL import Image

# Open the background and overlay images
background1 = Image.open(f'../tmp/gradio/c9e88903-0e15-41ce-836c-76c98d0c3d62/555f96d11f7ea45d54e42ad1d1c820c993a70f1f/image.png')
background2 = Image.open(f'../{result}/{directories[1]}/image.png')
overlay = Image.open('../content/resized_image1.png')

# You might need to resize the overlay or use it as is, depending on the need
# For example, resize overlay to match background's size
# overlay = overlay.resize(background.size, Image.ANTIALIAS)
print("Background mode:", background1.mode)
print("Overlay mode:", overlay.mode)
# Position the overlay at the desired position (top-left corner in this example)
# You can change (0, 0) to whatever coordinates you want
position = (0, 0)  # Position can be changed based on where you want to place the overlay
if background1.mode != 'RGBA':
    background = background1.convert('RGBA')
if background2.mode != 'RGBA':
    background = background2.convert('RGBA')
if overlay.mode != 'RGBA':
    overlay = overlay.convert('RGBA')
# Assuming both images are now RGBA
# Extract the alpha channel from the overlay
alpha_mask = overlay.split()[3]

# Paste the overlay using the alpha channel as a mask
background1.paste(overlay, (0, 0), alpha_mask)  # Adjust position as needed

# Save or show the result
background1.save('output_image11.png')  # Save as PNG to preserve transparency in the output
background1.show()

background2.paste(overlay, (0, 0), alpha_mask)  # Adjust position as needed

# Save or show the result
background2.save('output_image2.png')  # Save as PNG to preserve transparency in the output
background2.show()

Background mode: RGB
Overlay mode: RGBA


'/tmp/gradio/213e9684-a9d3-4ab8-92c9-a94409d767b2'